In [13]:
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [14]:
train = pd.read_csv('R:/IISc/Sem 2/PRNN/Assignments/assignment-1-data-sets/P1b_train_data_2D.txt', header=None)

In [15]:
D = 2

In [16]:
train = train.rename(columns={D: "y"})
train.head()

,0,1,y
0,0.1363,0.5861,1
1,1.5985,0.7568,1
2,-0.0179,0.4179,1
3,-2.7473,-0.9575,1
4,0.3327,1.1452,1


In [17]:
X = train.iloc[:, :D]
X = np.array(X)

In [18]:
gm = GaussianMixture(n_components=2, max_iter=10, random_state=0).fit(X)

In [19]:
gm.weights_

array([0.52341333, 0.47658667])

In [20]:
gm.means_

array([[ 2.96465286,  3.24850602],
       [-0.09560031, -0.04895932]])

In [21]:
gm.covariances_

array([[[ 0.98585216, -0.02931426],
        [-0.02931426,  2.95639524]],

       [[ 0.92316508,  0.49406501],
        [ 0.49406501,  0.96721445]]])

In [22]:
total = len(train.index)
p0 = gm.weights_[0]
p1 = gm.weights_[1]
mu0 = gm.means_[0]
mu1 = gm.means_[1]
sigma0 = gm.covariances_[0]
sigma1 = gm.covariances_[1]
mu0 = mu0.reshape((D, 1))
mu1 = mu1.reshape((D, 1))
sigma0 = sigma0.reshape((D, D))
sigma1 = sigma1.reshape((D, D))

In [23]:
def gaussian_prob(x, mu, sigma, d):
    xx = np.matmul(x-mu, np.linalg.inv(sigma))
    power = (-1/2)* np.matmul(np.matmul(x-mu, np.linalg.inv(sigma)), (x-mu).T)
    det = abs(np.linalg.det(sigma))
    value = (1/math.sqrt((2*math.pi)**d * det))* np.exp(power)
    return value[0][0]

def prediction(posterior_0, posterior_1):
    if posterior_0 > posterior_1:
        return -1
    else:
        return 1

In [24]:
train['posterior_0'] = train.apply(lambda row: p0 * gaussian_prob(np.array(row[0:D]), mu0, sigma0, D), axis=1)
train['posterior_1'] = train.apply(lambda row: p1 * gaussian_prob(np.array(row[0:D]), mu1, sigma1, D), axis=1)
train['predicted'] = train.apply(lambda row: prediction(row.posterior_0, row.posterior_1), axis=1)

In [25]:
train.head()

,0,1,y,posterior_0,posterior_1,predicted
0,0.1363,0.5861,1,3.025836e-04,0.073315,1
1,1.5985,0.7568,1,8.051957e-03,0.019857,1
2,-0.0179,0.4179,1,1.654858e-04,0.080108,1
3,-2.7473,-0.9575,1,1.869793e-10,0.001675,1
4,0.3327,1.1452,1,7.907570e-04,0.041170,1


In [26]:
y = train.iloc[:, [D]]
y_pred = train.iloc[:, [-1]]

In [27]:
predictions_train = np.array(y_pred) == np.array(y)

In [28]:
true_count = (predictions_train[:, 0] == True).sum()
false_count = (predictions_train[:, 0] == False).sum()
accuracy = true_count/(total) * 100
print(true_count, false_count)
print('Accuracy = ' + str(accuracy) + '%')

193 7
Accuracy = 96.5%


In [29]:
test = pd.read_csv('R:/IISc/Sem 2/PRNN/Assignments/assignment-1-data-sets/P1b_test_data_2D.txt', header=None)

In [30]:
test = test.rename(columns={D: "y"})
test.head()

,0,1,y
0,0.3208,-1.3389,1
1,-0.5736,0.2538,1
2,-1.0525,-1.3736,1
3,0.7437,-0.0372,1
4,-0.1212,1.1712,1


In [31]:
test['posterior_0'] = test.apply(lambda row: p0 * gaussian_prob(np.array(row[0:D]), mu0, sigma0, D), axis=1)
test['posterior_1'] = test.apply(lambda row: p1 * gaussian_prob(np.array(row[0:D]), mu1, sigma1, D), axis=1)
test['predicted'] = test.apply(lambda row: prediction(row.posterior_0, row.posterior_1), axis=1)

In [32]:
total = len(test.index)
y = test.iloc[:, [D]]
y_pred = test.iloc[:, [-1]]

In [33]:
predictions_test = np.array(y_pred) == np.array(y)

In [34]:
true_count = (predictions_test[:, 0] == True).sum()
false_count = (predictions_test[:, 0] == False).sum()
accuracy = true_count/(total) * 100
print(true_count, false_count)
print('Accuracy = ' + str(accuracy) + '%')

194 6
Accuracy = 97.0%
